In [10]:
import scipy.stats as stats
def Wilcoxon(big_data,col_name):
    col = big_data[col_name]
    labels = big_data[big_data.columns.values[len(big_data.T) -1]]
    ill = []
    healthy = []
    for i in range(len(col)):
        if labels[i] == 1:
            ill.append(col[i])
        else:
            healthy.append(col[i])
    #ill = [col[i] for i in range(len(col)) if labels[i] == 1 ]
    #healthy = [col[i] for i in range(len(col)) if labels[i] == 0 ]
    U = 0
    for i in ill:
        for h in healthy:
            if i < h:
                U += 1
            if i == h:
                U += 0.5
    n = len(ill)
    m = len(healthy)
    pvalue = stats.norm(loc = (n*m)/2 , scale = ((n*m)*(n+m+1)/12)**0.5).cdf(U)
    if pvalue > 0.5:
        pvalue = 1 - pvalue
    return pvalue


In [11]:
def wilxocon_for_all(data):
    col_names = data.columns.values[1:len(data.T) - 1]
    dict = {}
    for name in col_names:
        dict[name] = Wilcoxon(data, name)
    return dict

In [12]:
import pandas as pd
import numpy as np
#data = pd.read_csv("../HackathonMicrobiomeData/CAD/ClassCAD_train.csv")


#AT THE MOMENT THE FIRST COLOUMN CONTAINS LETTER SO REMOVE IT
#MAYBE THIS NEEDS TO BE CHANGED LATER

#THE FUNCTION CALCULATES A VALUE BETWEEN [0,1] FOR EVERY INDEPENDANT VARIABLE Xi, THE VALUE OF THE DEPENDANT VARIABLE Y IS NOT IN THE OUTPUT
#THE INDEPENDANT VARIABLE SHOULD ALWAYS BE IN THE LAST COLLUMN!!!!
#FOR THE CALCULATION OF THE VALUE FOR EVERY VARIABLE Xi WHICH HAS A HIGH CORRELATION WITH ANOTHER VARIABLE Xj -
#THE FORUMULAR IS |corXiY| - |corXiXj|/weight
#SO MAKE SURE TO TAKE THE RIGHT WEIGHT

def correlationPValues(data, weight):
    data = data.values
    data = data.T[1:len(data)].T

    for i in range(0, len(data)):
        if np.std(data[i]) == 0:
            print("Standardabweichung eines Attributs ist 0, Covarianz kann nicht bestimmt werden")

    covMatrix = np.cov(data.tolist())
    corMatrix = np.corrcoef(data.tolist())
    print(corMatrix)
    #get correlation from every independant variable with label
    correlationValues = corMatrix.T[data.shape[0] - 1]
    correlationWithLabel = corMatrix.T[data.shape[0] - 1]
    bestAttributes = []
    #add index of label to eliminatedAttributes
    eliminatedAttributes = [len(correlationValues)-1]

    variableToPValue = {}
    while(True):
        #get the highest correlation
        maxCor = -10
        bestAttribute = 0
        for i in range(0, len(correlationValues)):
            if i not in bestAttributes and i not in eliminatedAttributes:
                if correlationValues[i] > maxCor:
                    maxCor = correlationValues[i]
                    bestAttribute = i
        variableToPValue[bestAttribute] = maxCor
        #Add best index to best attributes
        bestAttributes.append(bestAttribute)
        #get correlation from every independant variable with best Attribute
        correlationWithBestAttribute = corMatrix.T[bestAttribute]

        #iterate through correlation and eliminate every variable with cor >= 0.7
        count = 0
        for i in range(0, len(correlationWithBestAttribute)):
            if not(i in bestAttributes or i in eliminatedAttributes):
                if correlationWithBestAttribute[i] >= 0.7:
                    count = count + 1
                    eliminatedAttributes.append(i)
                    variableToPValue[i] = np.maximum(np.abs(correlationWithLabel[i]) - np.abs(correlationWithBestAttribute[i])/weight, 0)

        if(len(correlationValues) == len(eliminatedAttributes) + len(bestAttributes)):
            break;
    return variableToPValue


In [13]:
#Calculates the cols to be deleted from the combination of Median and Covarianz filter
#please ensure to give two Dictionary with same size and no missing values...gaaarkein bock auf edge cases abfangen
def Efs(FilterMedian,FilterCov):

    #1. Calculate the importance values for the columns in Median

    minP = min(FilterMedian.values())
    medianImp = {}
    for col in FilterMedian.keys():
        impVal = 1 - FilterMedian[col] + minP
        medianImp[col] = impVal


    #2. Calculate the importance values for the columns in Cov

    maxBetha = max(FilterCov.values())
    covImp = {}
    for col in FilterCov.keys():
        impVal = FilterCov[col] / maxBetha
        covImp[col] = impVal


    #3. Combine the impVals
    combinedImp = {}
    for col in FilterMedian:
        combinedImp[col] = FilterMedian[col] + FilterCov[col]

    #4. Calculate the Cols to be deleted
    toDelete = []
    impMean = sum(combinedImp.values())/len(combinedImp)
    for col in combinedImp:
        if(combinedImp[col] <= impMean):
            toDelete.append(col)

    return toDelete



In [19]:
data = pd.read_csv("../../HackathonMicrobiomeData/CAD/ClassCAD_train.csv")

In [24]:
filterMedian = wilxocon_for_all(data)
print(filterMedian)
filterCov = correlationPValues(data, 3)
print(filterCov)
Efs(filterMedian,filterCov)

{'Bacteria;Abditibacteriota;Abditibacteria': 0.5, 'Bacteria;Acidobacteriota;Aminicenantia': 0.48396197026510523, 'Bacteria;Acidobacteriota;Blastocatellia': 0.49197936432162925, 'Bacteria;Actinobacteriota;Acidimicrobiia': 0.46697491324353363, 'Bacteria;Actinobacteriota;Actinobacteria': 0.00021494457137925238, 'Bacteria;Actinobacteriota;Coriobacteriia': 0.00032052417526742957, 'Bacteria;Actinobacteriota;Thermoleophilia': 0.48396197026510523, 'Bacteria;Bacteroidota;Bacteroidia': 0.03825549965822661, 'Bacteria;Bdellovibrionota;Bdellovibrionia': 0.40486169635117697, 'Bacteria;Bdellovibrionota;Oligoflexia': 0.4521836486696038, 'Bacteria;Campilobacterota;Campylobacteria': 0.4634668231439911, 'Bacteria;Chloroflexi;Anaerolineae': 0.49197936432162925, 'Bacteria;Cyanobacteria;Vampirivibrionia': 0.1798193285069336, 'Bacteria;Dependentiae;Babeliae': 0.49197936432162925, 'Bacteria;Desulfobacterota;Desulfovibrionia': 0.2946252314252801, 'Bacteria;Desulfobacterota;Desulfuromonadia': 0.4289714461731883

KeyError: 'Bacteria;Abditibacteriota;Abditibacteria'